# 随机梯度下降法
前面我们介绍了梯度下降法的数学原理，下面我们通过例子来说明一下随机梯度下降法。我们先自己实现，然后使用 pytorch 中自带的优化器

## 1. 实现SGD

随机梯度下降法非常简单，公式就是
$$
\theta_{i+1} = \theta_i - \eta \nabla L(\theta)
$$
非常简单，我们可以从 0 开始自己实现

In [ ]:
def sgd_update(parameters, lr):
    for param in parameters:
        param.data = param.data - lr * param.grad.data

In [ ]:
import numpy as np
import torch
from torchvision.datasets import MNIST # 导入 pytorch 内置的 mnist 数据
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt
%matplotlib inline

def data(x):
    x = np.array(x, dtype='float32') / 255 # 将数据变到 0 ~ 1 之间
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.reshape((-1,)) # 拉平
    x = torch.from_numpy(x)
    return x

def train(batch_size,lr):
    train_data = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    # 使用 Sequential 定义 3 层神经网络
    net = nn.Sequential(
        nn.Linear(784, 200),
        nn.ReLU(),
        nn.Linear(200, 10),
    )
    #net=net.cuda()
    # 开始训练
    losses = []
    idx = 0

    start = time.time() # 记时开始
    for e in range(1):
        train_loss = 0
        for im, label in train_data:
            im = Variable(im)#.cuda()
            label = Variable(label)#.cuda()
            # 前向传播
            out = net(im)
            loss = criterion(out, label)
            # 反向传播
            net.zero_grad()
            loss.backward()
            sgd_update(net.parameters(),lr) # 使用lr的学习率
            # 记录误差
            train_loss += loss.item()
            if idx % 30 == 0:
                losses.append(loss.item())
            idx += 1
        print('epoch: {}, Train Loss: {:.6f}'.format(e, train_loss / len(train_data)))
    end = time.time() # 计时结束
    print('使用时间: {:.5f} s'.format(end - start))
    x_axis = np.linspace(0, 5, len(losses), endpoint=True)
    plt.semilogy(x_axis, losses, label='batch_size=%d'%batch_size)
    plt.legend(loc='best')

train_set = MNIST('./data', train=True, transform=data, download=True) # 载入数据集，申明定义的数据变换
test_set = MNIST('./data', train=False, transform=data, download=True)

# 定义 loss 函数
criterion = nn.CrossEntropyLoss()

我们可以将 batch size 先设置为 1，看看训练效果

In [ ]:
train( batch_size=1,lr=1e-2)

可以看到，loss 在剧烈震荡，因为每次都是只对一个样本点做计算，每一层的梯度都具有很高的随机性，而且需要耗费了大量的时间

In [ ]:
train( batch_size=64,lr=1e-2)

通过上面的结果可以看到 loss 没有 batchsize=1时震荡那么剧烈，同时loss值也降到一定程度了，梯度相对而言更稳定。不过batch size 太大，对于内存的需求就更高，同时也不利于网络跳出局部极小点，所以现在普遍使用基于 batch 的随机梯度下降法，而 batch 的取值根据实际情况选择。

下面我们调高学习率，看看训练效果

In [ ]:
train( batch_size=64,lr=1)

可以看到，学习率太大会使得损失函数不断回跳，从而无法让损失函数较好降低，所以我们一般都是用一个比较小的学习率。

## 2. 使用Pytorch内置的SGD

pytorch 中已经为我们内置了随机梯度下降发，而且之前我们一直在使用，下面我们来使用 pytorch 自带的优化器来实现随机梯度下降

In [ ]:
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
# 使用 Sequential 定义 3 层神经网络
net = nn.Sequential(
    nn.Linear(784, 200),
    nn.ReLU(),
    nn.Linear(200, 10),
)
net=net#.cuda()
optimzier = torch.optim.SGD(net.parameters(), 1e-2)

# 开始训练
start = time.time() # 记时开始
for e in range(1):
    train_loss = 0
    for im, label in train_data:
        im = Variable(im)#.cuda()
        label = Variable(label)#.cuda()
        # 前向传播
        out = net(im)
        loss = criterion(out, label)
        # 反向传播
        optimzier.zero_grad()
        loss.backward()
        optimzier.step()
        # 记录误差
        train_loss += loss.item()
    print('epoch: {}, Train Loss: {:.6f}'
          .format(e, train_loss / len(train_data)))
end = time.time() # 计时结束
print('使用时间: {:.5f} s'.format(end - start))